# BME 590 - Workshop 3 - Moving Labware
**Professor:** Emma Chory, Ph.D.

**Authors:** 
Rick Wierenga, Joe Laforet, Stefan Golas, Ben Perry

---

### Usage Note
**Reminder** - You should be running this notebook **locally** on **VS Code** not navigating it through **GitHub**.

**Reminder 2** - Remember to run `git pull` before copying this notebook so you get the most recent class updates. See [Section 6 of the class README](https://github.com/chory-lab/bme590-fall-2025#step-6-updating-to-the-latest-version)

---

### Moving Labware
Now that we have covered moving liquids around on the liquid handling apparatus, we hit a bottleneck: we are limited by the labware we can place on a given deck! If only there were a way to be able to automatically move labware to and from the liquid handling deck, we could:

- Replace tip racks when they run out of tips with a fresh box.

- Throw used plates into the trash.

- Rotate labware

- Move plates to/from specific locations on the deck that may contain specialized modules, such as heater-shakers, strong magnets, etc.

- Move plates to another piece of equipment outside the deck, such as a refrigerator, plate reader, or even another liquid handling deck (2nd robot).

To actually move labware, things get a bit free-form, since there isn't a singular way to do it. Some robotic platforms, such as [Hamilton Microlab STAR platform](https://www.hamiltoncompany.com/microlab-star?srsltid=AfmBOor9rRfwPDsMrefQ3RGW64R1prQ03U29WxRBeRFfOGEDS3UAVOk3) or [OpenTrons-2](https://opentrons.com/products/opentrons-flex-gripper-gen1?kw=&cpn=21988025885&utm_term=&utm_source=google&utm_medium=cpc&utm_campaign=NAMER_Pmax_Brand_Plus_Emea&hsa_cam=21776817541&hsa_grp=&hsa_mt=&hsa_src=x&hsa_ad=&hsa_acc=2303351826&hsa_net=adwords&hsa_kw=&hsa_tgt=&hsa_ver=3&gad_source=1&gad_campaignid=21981643080&gbraid=0AAAAADemOsUxn_dtGtEETQnNTZDQP95mg&gclid=Cj0KCQjw3OjGBhDYARIsADd-uX5NmfCoeBmgGsdW_Vi9PovGUjK89S0KfF5dOU00o2EUQ4l_zZeNuucaAug4EALw_wcB) have grippers that are built in or can be attached to perform **intra-deck moves**, meaning moving labware from one spot on the deck to another.

For even more flexibiility, there are **robot arm** systems which can move plates among **different lab equipment**, such as moving a plate to a plate reader, PCR machine, refrigerator, or other equipment, such as the Peak Robotics [KX2-500](https://peakrobotics.com/product/kx2-500/). Sometimes, these arms may be attached to **linear rail** systems, like [this one](https://peakrobotics.com/product/linear-rail-500mm/) to even more the arm around.

Even more flexible are humanoid robots such as [Figure](https://www.figure.ai/) or [Unitree](https://www.unitree.com/R1) that can perform many tasks in a lab, especially when powered with open source robotic world models like [LeRobot](https://github.com/huggingface/lerobot) and NVIDIA [Isaac GROOT](https://github.com/NVIDIA/Isaac-GR00T).

For this tutorial, we will focus on **intra-deck grippers** since they are the simplest to conceptually understand. While there are some implementations of control over gripper functions in [PLR](https://docs.pylabrobot.org/user_guide/00_liquid-handling/hamilton-star/iswap-module.html#rotations), for the most part we will be implementing **custom classes to siimulate gripper function**.

If you are using an extenral arm, robot, or other device, you will ultimately **integrate the code to control that device** with your liquid handling code, and this notebook will give you good practice at that.

In this notebook we will:

- Set up an OpenTrons-2 Deck with a custom Gripper class that we implement from scratch.

- Demonstrate examples of using this gripper class to move plates around and add/remoe a custom **plate lid** to each piece of labware.

- Task you with several exercises at implementing your own Gripper and functionalities.

### Initial OT-2 Deck Setup & Imports

First, let's go ahead and import the equipment we need.

In [ ]:
# standard imports
from pylabrobot.liquid_handling.backends.backend import LiquidHandlerBackend
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import LiquidHandlerChatterboxBackend
from pylabrobot.resources.opentrons import OTDeck
from pylabrobot.visualizer.visualizer import Visualizer

import time

# resources for deck setup
from pylabrobot.resources import (
    Deck,
    Resource,
    set_tip_tracking,
    set_volume_tracking,
    set_cross_contamination_tracking,
    corning_96_wellplate_360ul_flat,
    opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap_acrylic,
    opentrons_96_tiprack_1000ul
)

Enable trackers for error catching

In [ ]:
set_tip_tracking(enabled = True)
set_volume_tracking(enabled = True)
set_cross_contamination_tracking(enabled = True)

Define the `visualize_deck` function

In [ ]:
async def visualize_deck(deck: Deck,
                         backend: LiquidHandlerBackend):
    # try setting up the deck with error-catching
    try:
        lh = LiquidHandler(backend=backend, deck=deck)
        vis = Visualizer(resource = lh)
        await lh.setup()
        await vis.setup()
        return lh
    except Exception as e:
        print(f"Error! Got excpetion: {e}")

Create the OT-2 Deck

In [ ]:
async def make_same_ot2():

    # instantiate deck
    deck = OTDeck()

    # add tip racks
    tip_rack_slots = [10, 11]
    for i, tip_rack_slot in enumerate(tip_rack_slots):
        deck.assign_child_at_slot(opentrons_96_tiprack_1000ul(name = f"tip_rack_{i}"), tip_rack_slot)
    
    # add 96 well plate and tube rack
    deck.assign_child_at_slot(corning_96_wellplate_360ul_flat(name = f"plate"), 5)
    tube_rack = opentrons_24_tuberack_eppendorf_2ml_safelock_snapcap_acrylic(name = "tube_rack")
    deck.assign_child_at_slot(tube_rack, 2)

    # fill tube rack diagonals with liquid
    dye = ("Red Dye", 2_000)
    for i, well_batch in enumerate(tube_rack.traverse(batch_size = 1, start = "top_left", direction = "snake_down")):
        if i % 2 == 0:
            for well in well_batch:
                well.tracker.set_liquids([dye])
    return deck

# call function
deck = await make_same_ot2()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

Let's print the deck summary

In [ ]:
print(deck.summary())

### Introducing the OT-2 Gripper!

Various liquid handling robots have a part called a Gripper. It is essentially an arm that you can program to move labware around the deck as your protocol progresses. The OpenTrons OT2 does not have one of these, but the OpenTrons Flex does. For this tutorial, we will be using a simulated Gripper class that implements the methods we need to demonstrate how you would use this feature in a real experiment.

To do this, we will implement a Python class with two methods:

- `__init__()`: This function is a special python **dunder** method which tells Python how to **instantiate the class**.

- `move_labware()`: This function will simply take in a certain labware on the deck and move it to a target desination spot.

Let's code it below!

In [ ]:
# to dfine a class, use class <class_name>: format
class Gripper:

    def __init__(self, lh: LiquidHandler):
        """
        Constructor, this function instantiates our Gripper, given only one input argument, the liquid handler head.
        
        self - Argument required by Python class, signals a method is part of a class and has all atributes of self
        lh - Liquid handler object, to be stored in self.lh, which is accessible by all class methods
        """
        self.lh = lh # assign a property of the class, which will be accessible by all methods of the class

    def move_labware(self, labware: Resource, dest_spot: int):
        """
        Basic move labware function for OT-2, which will move a labware from its curent spot to a new spot.

        self - Argument required by all methods in Python class, signals method is part of the class.
        labware - The specific labware to move on the OT-2 deck. Must exist on deck
        dest_spot - The target destination spot on the OT-2 deck to move the labware to.
        """
        self.lh.deck.unassign_child_resource(labware)
        self.lh.deck.assign_child_at_slot(labware, dest_spot)
        print(f"Moving {labware.name} to spot {dest_spot}")

Ok great, now let's try instantiating the class with out existing liquid handler object.

In [ ]:
ot_gripper = Gripper(lh)

Great! Now let's try to use it to move the tube rack to spot 1. First get the tube rack

In [ ]:
tube_rack = deck.get_resource("tube_rack")

Move it using the pre-defined `move_labware` function

In [ ]:
ot_gripper.move_labware(tube_rack, dest_spot = 1)

Let's print the deck summary. **You should see that the tube rack has moved**.

Check the visualizer too!

In [ ]:
print(deck.summary())

We can even write a for loop to move the tube rack up in a vertical column. Let's try that

In [ ]:
# for each slot available
for i in range(4, 11, 3):
    # check to see if the deck slot in occupied
    if deck.slots[i - 1] is None:
        ot_gripper.move_labware(tube_rack, dest_spot = i)
        print(deck.summary())
        time.sleep(0.5)

We can make this more general by adding a start_slot number to move up a column. Let's reset where the tube rack is and try again.

In [ ]:
# define a start slot so we can start from anywhere on the deck
start_slot = 4
ot_gripper.move_labware(tube_rack, dest_spot = start_slot) # move the labware to start slot, if it elsewhere. for visualization

# delay -> switch to visualizer here
time.sleep(0.5)

# for each slot in a column
for i in range(start_slot + 3, 12, 3):
    if deck.slots[i - 1] is None:
        ot_gripper.move_labware(tube_rack, dest_spot = i)
        print(deck.summary())
        time.sleep(0.5)

Great! Turns out, we can also figure out, based on the labware, what the **start slot** is!

In [ ]:
# show that can can determine whether start slot is 4
ot_gripper.move_labware(tube_rack, dest_spot = 4)
start_slot = deck.slots.index(tube_rack) + 1
print(f"Start slot is: {start_slot}")

# or whether it is 1
ot_gripper.move_labware(tube_rack, dest_spot = 1)
start_slot = deck.slots.index(tube_rack) + 1
print(f"Start slot is: {start_slot}")

We can now replace our start_slot variable with this one to enable referencing by labware only.

In [ ]:
# Now determine the start slot automatically, given the labware
start_slot = deck.slots.index(tube_rack) + 1
ot_gripper.move_labware(tube_rack, dest_spot = 1)

# delay -> switch to visualizer here
time.sleep(0.5)

# same as before. iterate over column slots
for i in range(start_slot + 3, 12, 3):
    print(i)
    if deck.slots[i - 1] is None:
        ot_gripper.move_labware(tube_rack, dest_spot = i)
        print(deck.summary())
        time.sleep(0.5)

Let's now turn this into a **function** so we don't have to keep copy/pasting this code!

In [ ]:
# define function version of our column code
def move_labware_column_up(labware: Resource):

    # find start index
    start_slot = deck.slots.index(labware) + 1
    ot_gripper.move_labware(labware, dest_spot = 1)

    # delay to swap to visualizer
    time.sleep(0.5)

    # iterate over slots and move labware
    for i in range(start_slot + 3, 12, 3):
        print(i)

        # make sure target slot is not already occupied
        if deck.slots[i - 1] is None:
            ot_gripper.move_labware(labware, dest_spot = i)
            print(deck.summary())
            time.sleep(0.5)

Now we can call the function to move the plate again. Let's reset the plate position to 1.

Call our new function to move the plate up to the last unoccupied spot.

In [ ]:
move_labware_column_up(tube_rack)

This seems like a useful utility to add to our OT-2 gripper. Let's add it as a class method.

By **defining the function** within the class, we can **use it later by just initializing the class**

In [ ]:
class NewGripper:

    def __init__(self, lh: LiquidHandler):
        """
        Constructor, this function instantiates our Gripper, given only one input argument, the liquid handler head.
        
        self - Argument required by Python class, signals a method is part of a class and has all atributes of self
        lh - Liquid handler object, to be stored in self.lh, which is accessible by all class methods
        """
        self.lh = lh # assign a property of the class, which will be accessible by all methods of the class

    def move_labware(self, labware: Resource, dest_spot: int):
        """
        Basic move labware function for OT-2, which will move a labware from its curent spot to a new spot.

        self - Argument required by all methods in Python class, signals method is part of the class.
        labware - The specific labware to move on the OT-2 deck. Must exist on deck
        dest_spot - The target destination spot on the OT-2 deck to move the labware to.
        """
        self.lh.deck.unassign_child_resource(labware)
        self.lh.deck.assign_child_at_slot(labware, dest_spot)
        print(f"Moving {labware.name} to spot {dest_spot}")

    def move_labware_column_up(self, labware: Resource):
        """
        Move function to move a given labware up a column

        self - Argument required by all methods in Python class, signals method is part of the class.
        labware - The specific labware to move on the OT-2 deck. Must exist on deck
        """
        start_slot = self.lh.deck.slots.index(labware) + 1
        for i in range(start_slot + 3, 12, 3):
            print(f"Moving {labware.name} to spot {i}")
            if deck.slots[i - 1] is None:
                self.move_labware(labware, dest_spot = i) # NOTICE - See how we can use the method already defined by the class to move the labware? This means we don't have to re-write the code.
                time.sleep(0.5)
            else:
                print(f"Spot {i} occupied! Skipping...")

Great, now let's do the same move again, but with our new class.

In [ ]:
deck = await make_same_ot2()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

tube_rack = deck.get_resource("tube_rack")

In [ ]:
# define new gripper and use it to move in a column upwards.
ot_new_gripper = NewGripper(lh)
ot_new_gripper.move_labware(tube_rack, 1)
ot_new_gripper.move_labware_column_up(tube_rack)

Great, now experiment a bit by using the following cell to try and move labware around the deck:

- Try to move the plate to slot 3 then move it up to slot 9 only in two function calls.

- Try to move the tube rack to slot 1 then slot 5 then slot 7 with a for loop

In [ ]:
# Test code here
labware_to_move = ...
ot_new_gripper.move_labware(...)

### Adding Functions - Moving Labware by Name

Notice how we still have to **first define the labware we are moving** by using `get_resoorce()`? That seems a bit annoying. Perhaps we can add it as a class method to our existing class and replace the old functions so they can access things by labware name alone. Let's try.

In [ ]:
class NewNamedGripper:

    def __init__(self, lh: LiquidHandler):
        """
        Constructor, this function instantiates our Gripper, given only one input argument, the liquid handler head.
        
        self - Argument required by Python class, signals a method is part of a class and has all atributes of self
        lh - Liquid handler object, to be stored in self.lh, which is accessible by all class methods
        """
        self.lh = lh # assign a property of the class, which will be accessible by all methods of the class

    def move_labware(self, labware: Resource, dest_spot: int):
        """
        Basic move labware function for OT-2, which will move a labware from its curent spot to a new spot.

        self - Argument required by all methods in Python class, signals method is part of the class.
        labware - The specific labware to move on the OT-2 deck. Must exist on deck
        dest_spot - The target destination spot on the OT-2 deck to move the labware to.
        """
        self.lh.deck.unassign_child_resource(labware)
        time.sleep(0.5)
        self.lh.deck.assign_child_at_slot(labware, dest_spot)
        print(f"Moving {labware.name} to spot {dest_spot}")

    def move_labware_column_up(self, labware: Resource):
        """
        Move function to move a given labware up a column

        self - Argument required by all methods in Python class, signals method is part of the class.
        labware - The specific labware to move on the OT-2 deck. Must exist on deck
        """
        start_slot = self.lh.deck.slots.index(labware) + 1
        for i in range(start_slot + 3, 12, 3):
            print(f"Moving {labware.name} to spot {i}")
            if self.lh.deck.slots[i - 1] is None:
                self.move_labware(labware, dest_spot = i) # NOTICE - See how we can use the method already defined by the class to move the labware? This means we don't have to re-write the code.
                time.sleep(0.5)
            else:
                print(f"Spot {i} occupied! Skipping...")
    
    def get_labware_by_name(self, name: str):
        """
        Function to retrieve labware on the deck by the resource name

        self - Argument required by all methods in Python class, signals method is part of the class.
        name - The specific labware name to retrieve.
        """
        return self.lh.deck.get_resource(name)

    def move_labware_by_name(self, labware_name: str, dest_spot: int):
        """
        Move function to move a given labware to a destination spot, by name.

        self - Argument required by all methods in Python class, signals method is part of the class.
        labware_name - The specific name of the labware to move on the OT-2 deck. Must exist on deck
        dest_spot - The target destination spot on the OT-2 deck to move the labware to.
        """
        labware = self.get_labware_by_name(labware_name)
        self.move_labware(labware = labware, dest_spot = dest_spot)

    def move_labware_column_up_by_name(self, labware_name: str):
        """
        Move function to move a given labware up a column by labware name.

        self - Argument required by all methods in Python class, signals method is part of the class.
        labware_name - The specific name of the labware to move on the OT-2 deck. Must exist on deck
        """
        labware = self.get_labware_by_name(labware_name)
        self.move_labware_column_up(labware = labware)

Great, now let's try using this new gripper that can access things by name alone!

In [ ]:
deck = await make_same_ot2()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

# define new gripper
named_gripper = NewNamedGripper(lh)
named_gripper.move_labware_by_name("tube_rack", 1)
named_gripper.move_labware_column_up_by_name("tube_rack")

This does seem a bit tedious, redefining the class each time we want to add functionality.

Luckily, in Python, we can implement `subclasses` that **inherit the methods of the previous class**. For example, we could define the same gripper as above by the following code:

In [ ]:
class NewNamedGripperInherited(NewGripper): # NOTICE - See how we inherit by simply calling ChildClass(ParentClass)? Now we don't have to re-define all the old methods!

    # By inheriting the NewGripper class, we get all the following methods:
    # __init__
    # move_labware
    # move_labware_column_up
    # See how the functions below don't need to re-define these ones? That's the power of inheritance

    def get_labware_by_name(self, name: str):
        """
        Function to retrieve labware on the deck by the resource name
        """
        return self.lh.deck.get_resource(name)

    def move_labware_by_name(self, labware_name: str, dest_spot: int):
        """
        Move function to move a given labware to a destination spot, by name.
        """
        labware = self.get_labware_by_name(labware_name)
        self.move_labware(labware = labware, dest_spot = dest_spot)

    def move_labware_column_up_by_name(self, labware_name: str):
        """
        Move function to move a given labware up a column by labware name.
        """
        labware = self.get_labware_by_name(labware_name)
        self.move_labware_column_up(labware = labware)

This is a lot more concise, but let's check that it can achieve the **same functionality** as before.

In [ ]:
deck = await make_same_ot2()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

# define new gripper with inheritance
named_gripper = NewNamedGripperInherited(lh)
named_gripper.move_labware_by_name("tube_rack", 1)
named_gripper.move_labware_column_up_by_name("tube_rack")

### Adding Functions - Checking if Labware exists
What happens if we call the `move_labware_by_name()` function with a name of an item that does not exist on the deck?

In [ ]:
# this should throw an error if name incorrect
named_gripper.move_labware_by_name("bad_name", dest_spot = 3)

We should get an error here! We should probably update the `get_labware_by_name()` function to include a check for if the labware exists on the deck in the first place and print it to the console with some more information to help us de-bug better.

Ideally, we could print the current deck layout if a given labware name isn't found, so we can see what is on the deck.

Fortunately, we don't have to **re-define** the entire class, we can simply inherit the class above, and **override** the function of interest by re-defining it with the same name. Let's try this below.

In [ ]:
class FixedGripper(NewNamedGripperInherited): # NOTICE - See how we inherit by simply calling ChildClass(ParentClass)? Now we don't have to re-define all the old methods!

    # By inheriting the NewGripper class, we get all the following methods:
    # __init__
    # move_labware
    # move_labware_column_up
    # move_labware_by_name
    # move_labware_column_up_by_name
    # and, importantly, get_labware_by_name

    # let's re-define this function with our corrected code.
    def get_labware_by_name(self, name: str):
        """
        Function to retrieve labware on the deck by the resource name
        """
        if not self.lh.deck.has_resource(name):
            raise ValueError(F"Resource: {name} not found on deck! See deck sumamry below: \n\n {self.lh.deck.summary()}")
        else:
            return self.lh.deck.get_resource(name)

Let's make sure it still works

In [ ]:
# define new gripper with inheritance
fixed_gripper = FixedGripper(lh)
fixed_gripper.move_labware_by_name("tube_rack", 1)
fixed_gripper.move_labware_column_up_by_name("tube_rack")

Now let's try that erroneous code earlier and see what happens now.

In [ ]:
# this should throw our better error -> remember you can use this to do other actions if errors would occur!
fixed_gripper.move_labware_by_name("bad_name", dest_spot = 3)

Now our new error

```txt
alueError: Resource: bad_name not found on deck! See deck sumamry below: 

 
Deck: 624.3mm x 565.2mm

+-----------------+-----------------+-----------------+
|                 |                 |                 |
| 10: tip_rack_0  | 11: tip_rack_1  | 12: trash_co... |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  7: tube_rack   |  8: Empty       |  9: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  4: Empty       |  5: plate       |  6: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|  1: Empty       |  2: Empty       |  3: Empty       |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
```

is more interpretable!

### Introducing the Lid!

Some plates in molecular biology research are sold with [lids](https://www.celltreat.com/product/229590/?srsltid=AfmBOoqa9Yomsz66CAb_Z2hs2oeQ7s8MMfNr-brQlm15lLeJ4FGTj10j) and these are actually supported in PLR. For example, this particular plate has a lid modeled in the [source code](https://github.com/PyLabRobot/pylabrobot/blob/7234de0ae521c8be9cd7b52cdc881eb3bdb82254/pylabrobot/resources/celltreat/plates.py#L76)

The lids are described by the `Lid` class in PLR [source](https://github.com/PyLabRobot/pylabrobot/blob/7234de0ae521c8be9cd7b52cdc881eb3bdb82254/pylabrobot/resources/plate.py#L26) and are simply assigned to a plate via the following code in the [source](https://github.com/PyLabRobot/pylabrobot/blob/7234de0ae521c8be9cd7b52cdc881eb3bdb82254/pylabrobot/resources/plate.py#L106):

```python
if lid is not None:
    self.assign_child_resource(lid)
```

What if had a gripper that was able to **add and remove lids** through some separate code (Python or otherwise)? We could actually write some methods for our gripper that **add or remove the lid** and then **tell PLR about it**

This is getting a bit into the nuance of PLR, but the basic gist is:

- If you are doing an operation on a liquid handling deck that **is not covered in PLR** you need to tell **PLR about it**.

- You have actually already done this with setting up liquid reservoirs. The act of manually filling a reservoir with solvent or dye and placing on the deck is **not tracked automatically by PLR**; however, once it is placed on the deck, you tell PLR about it, then **PLR can track it with the standard liquid handling protocols, volume/cross-contamination checking, etc.**

- The same thing is true for plate lids. If we have a gripper or piece of equipment not covered by PLR but accessible, we should **tell PLR about the lid being present or not**

Let's go ahead and define a custom plate lid for the OpenTrons plate.

**Note** - The units are in millimeters (mm) and can often be found on **vendor websites** for a given plate.

In [ ]:
from pylabrobot.resources.plate import Lid

# define a lid compatible with the 96-well plate on OT-2
plate_lid = Lid(name = "Lid", # name - needs to be unique on a given deck
                size_x = 127.0, # mm
                size_y = 86.0, # mm
                size_z = 0, # z height of lid itself -> default to 0
                nesting_z_height = 1, # added height to overall plate, when covered by lid
                model = "Cos_96_FL_Lid")

Now, let's take our `FixedGripper` and add some functionalities to tell PLR that we have added or removed a lid from the plate in question.

In [ ]:
class LidGripper(FixedGripper): # NOTICE - See how we inherit by simply calling ChildClass(ParentClass)? Now we don't have to re-define all the old methods!

    # function to add lid from an external source
    def add_external_lid(self, plate_name: str, lid: Lid): 
        plate = self.get_labware_by_name(plate_name)
        plate.assign_child_resource(lid)
        print(f"Adding lid {lid.name} to plate {plate.name}")

    # function to simulate throwing lid away to external trash can
    def remove_lid_to_trash(self, plate_name: str):
        plate = self.get_labware_by_name(plate_name)
        assert plate.has_lid(), "Plate doesn't have a lid!"
        lid = plate.lid
        print(f"Removing lid from {plate.name}")
        plate.unassign_child_resource(lid)

Try adding the plate_lid

In [ ]:
deck = await make_same_ot2()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

# define lid gripper
lid_gripper = LidGripper(lh)

# add lid
lid_gripper.add_external_lid(plate_name = "plate", lid = plate_lid)

# go look at visualizer!

Now let's simulate removing the lid and moving it to the trash.

In [ ]:
# add lid
lid_gripper.remove_lid_to_trash(plate_name = "plate")

What do you think happens if we try to add a lid to plate that already has a lid?

In [ ]:
lid_gripper.add_external_lid(plate_name = "plate", lid = plate_lid)
lid_gripper.add_external_lid(plate_name = "plate", lid = plate_lid)

This demonstrates the functionality of PLR tracking lids works when you tell it a lid is present!

### Exercises

---

**TO-DO:** For each of the following exercses, since they are a bit different from each other, we will require submission of one or more of the following:

- a `.txt` file containing your code.

- a `.gif` file containing a GIF animation of your protocol running

- a `.png` or `.jpg` image of your deck setup, if needed.

We will explicitly tell you for each exercise and sub-exercise, which items to submit. There will be a **sample submission format** for each exercise.

---

Some exercises below will ask you to define your own **functions or classes**. We will provide the **function or class name** and sometimes the **input argument names** for you, but in gneeral, the body of the functions is up to you.

You should include `time.sleep(x)` calls between every step so you have time to visualize the protocol as it runs. At a minimum, `x = 0.1` for 0.1 s delay. Experiment with this value for one that works for you.

Once you get your protocol working as intended for each problem, you will need to **record a GIF** of your protocol running for each exercise, as directed

**IMPORTANT** - You should be judiciously commenting your code to explain its function. We will grade every problem by **quality of code**. Excessively long code or lack of comments will be subject to **point deduction**

Furthermore, you can write the code how you see fit. However, **do not change function names** and make sure to **include your imports** at the top of your .txt file submission.

---

### **Exercise 1.** Enhancing Exising Gripper for OT-2 Operations (40 pts)


**1.A.** Add Plate

First, let's set up an empty OT-2 Deck and a base `Exercise_1_Gripper` class.

In [ ]:
# DO NOT MODIFY
deck = OTDeck()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())


class Exercise_1_Gripper:

    def __init__(self, lh: LiquidHandler):
        """
        Constructor, this function instantiates our Gripper, given only one input argument, the liquid handler head.
        
        self - Argument required by Python class, signals a method is part of a class and has all atributes of self
        lh - Liquid handler object, to be stored in self.lh, which is accessible by all class methods
        """
        self.lh = lh # assign a property of the class, which will be accessible by all methods of the class

    def move_labware(self, labware: Resource, dest_spot: int):
        """
        Basic move labware function for OT-2, which will move a labware from its curent spot to a new spot.

        self - Argument required by all methods in Python class, signals method is part of the class.
        labware - The specific labware to move on the OT-2 deck. Must exist on deck
        dest_spot - The target destination spot on the OT-2 deck to move the labware to.
        """
        self.lh.deck.unassign_child_resource(labware)
        self.lh.deck.assign_child_at_slot(labware, dest_spot)
        print(f"Moving {labware.name} to spot {dest_spot}")

    def get_labware_by_name(self, name: str):
        """
        Function to retrieve labware on the deck by the resource name
        """
        return self.lh.deck.get_resource(name)

    def move_labware_by_name(self, labware_name: str, dest_spot: int):
        """
        Move function to move a given labware to a destination spot, by name.
        """
        labware = self.get_labware_by_name(labware_name)
        self.move_labware(labware = labware, dest_spot = dest_spot)

Now, implement the `add_plate()` function which should input a `slot` number and a `plate_name` and add it to the deck. For this exercise, use only `corning_96_wellplate_360ul_flat` plates

Comment your code

In [ ]:
# imports
from pylabrobot.resources import ... # YOUR CODE HERE

class Exercise_1A_Gripper(Exercise_1_Gripper):
    def add_plate(self, slot: int, plate_name: str):
        ... # YOUR CODE HERE

Now using `add_plate()` add three plates to the OT-2 deck using **4 lines of Python or less (excluding comments and blank lines)**:

1. Name = "plate_1", Slot = 1

2. Name = "plate_2", Slot = 4

3. Name = "plate_3", Slot = 7

4. Name = "plate_X", Slot = 10

Comment your code

In [ ]:
gripper = Exercise_1A_Gripper(lh = lh)

# YOUR CODE HERE

**1.B.** Add Reservoir

Great! Now we need a function to add a reservoir to the deck. Create a class method called `add_reservoir` that inputs:

- `slot` - The slot for the reservoir

- `reservoir_name` - The name of the reservoir

- `liquid_name` - The name of the liquid in the reservoir

- `liquid_vol` - The volume of liquid in the reservoir, in microliters

For this function, use `axygen_1_reservoir_90ml` and assume we are not using a pre-defined `Liquid` class.

**Note -** The plates are often characterized by vendor name; for example, this one refers to the [Axygen reservoir](https://www.avantorsciences.com/ca/en/product/4694740/axygen-single-and-multi-well-reservoirs-corning)

To get you used to navigating resources in PLR, **find another reservoir that can server the purpose below (90+ mL)** and put it's import in the spot dubbed `## YOUR RESERVOIR IMPORT HERE`

You should have **two imports** - one for the Axygen reservoir that you will use, and one other appropriate reservoir. Use the axygen one in your method.

Comment your code.

In [ ]:
# imports
from pylabrobot.resources import ... # YOUR CODE HERE

# other import should have at least 90 mL
from pylabrobot.resources import ... ## YOUR RESERVOIR IMPORT HERE


class Exercise_1B_Gripper(Exercise_1_Gripper):
    def add_reservoir(self,
                      slot: int, 
                      reservoir_name: str,
                      liquid_name: str,
                      liquid_vol: str):
        ... # YOUR CODE HERE

Great! Now use your function to add reservoirs of the following liquids to the deck:

- Slot = 2, Reservoir Name = "water_reservoir", Liquid = "water", Volume = 10 mL

- Slot = 5, Reservoir Name = "ethanol_reservoir", Liquid = "ethanol", Volume = 20.211 mL

- Slot = 8, Reservoir Name = "bleach_reservoir", Liquid = "bleach", Volume = 80 mL

Do it in **6 lines of code or less (excluding comments and blank spaces).** Comment your code

In [ ]:
gripper = Exercise_1B_Gripper(lh = lh)
... # YOUR CODE HERE

**1.C** Move Plate Row-Wise

Great! Now remember that function we wrote earlier to move labware by columns? You should now implement a function that moves plates row-wise that takes in e following arguments:

- `labware_name` - Name of labware to move

- `direction` - Either `left` or `right`, represents the direction to move the plates 

Remember, the function should skip over any plates that are in the way. **Note -** You should check that your function works from **any starting slot number**. The row version may not be as simple as the column version.

Comment your code

In [ ]:
class Exercise_1C_Gripper(Exercise_1_Gripper):
    def move_labware_row_wise(self,
                            labware_name: str,
                            direction: str):
        ... # YOUR CODE HERE

Great! Now use that function to move all the **96-well plates** to the right first, then all the **reservoirs** to the left after that.

Do it in **six lines of code or less (excluding comments and blank spaces)**

Comment your code

In [ ]:
gripper = Exercise_1C_Gripper(lh = lh)
... # YOUR CODE HERE

---

**TO-DO:** Submission Instructions. In the cell below, we provide a template for your .txt file submission. Please copy-paste the relevant parts of your code into the designated sections below and submit as a .txt file named `exercise_1.txt`

**TO-DO:** Please run your code, starting from the **blank deck** all the way through the movement of the 96-well plates to the right and reservoirs to the left, record the GIF, and submit as a .gif file named `exercise_1.gif`

---

In [ ]:
# Exercise 1. All Imports
# --- YOUR CODE HERE ---

## Exercise 1.B. Reservoir Import
## YOUR RESERVOIR IMPORT HERE


# CLASS METHODS SECTION
# Exercise 1.A. Add Plate
class Exercise_1A_Gripper(Exercise_1_Gripper):
    def add_plate(self, slot: int, plate_name: str):
        ... # YOUR CODE HERE

# Exercise 1.B. Add Reservoir
class Exercise_1B_Gripper(Exercise_1_Gripper):
    def add_reservoir(self,
                      slot: int, 
                      reservoir_name: str,
                      liquid_name: str,
                      liquid_vol: str):
        ... # YOUR CODE HERE

# Exercise 1.C. Move Labware Row-wise
class Exercise_1C_Gripper(Exercise_1_Gripper):
    def move_labware_row_wise(self,
                            labware_name: str,
                            direction: str):
        labware = self.get_labware_by_name(labware_name)

# CODE BLOCKS SECTION
# Exercise 1.A. Add Plate
gripper = Exercise_1A_Gripper(lh = lh)
... # YOUR CODE HERE

# Exercise 1.B. Add Reservoir
gripper = Exercise_1B_Gripper(lh = lh)
... # YOUR CODE HERE

# Exercise 1.C. Move Labware Row-wise
gripper = Exercise_1C_Gripper(lh = lh)
... # YOUR CODE HERE

# Save as exercise_1.txt and submit. Don't forget to submit exercise_1.gif as well.

### **Exercise 2.** Create a new Gripper for the Hamilton STAR (60 pts)

Now that you have seen the functionality for a custom Gripper on the OpenTrons-2 deck, you will now create your own custom gripper for the Hamilton STAR deck. Let's start with a simple deck layout

In [ ]:
# DO NOT MODIFY

# imports
from pylabrobot.resources import STARDeck, PLT_CAR_L5AC_A00, corning_96_wellplate_360ul_flat

# function to define custom deck
def make_deck():

    # initialize deck
    deck = STARDeck()

    # add plates carriers and plates
    plate_carrier_0 = PLT_CAR_L5AC_A00("plate_carrier_0")
    plate_carrier_1 = PLT_CAR_L5AC_A00("plate_carrier_1")
    for i in range(5):
        plate_carrier_0[i] = corning_96_wellplate_360ul_flat(name = f"plate_{i}")
    
    # add carriers to deck
    deck.assign_child_resource(plate_carrier_0, rails = 7)
    deck.assign_child_resource(plate_carrier_1, rails = 14)

    return deck

# create deck
deck = make_deck()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

**2.A.** Initial Gripper Class

Now let's go ahead and define a base class with an `__init__` function and a function to `get_labware_by_name()`

Comment your code

In [ ]:
class Exercise_2A_Gripper:
    def __init__(self, lh: LiquidHandler):
        ... # YOUR CODE HERE

    def get_labware_by_name(self, name: str):
        ... # YOUR CODE HERE

**2.B.** Plate Mover

Implement a function named `move_plates()` that takes the following arguments:

- `input_carrier_name` - String identifier for the carrier containing the plate to move.

- `plate_name` - String identifier of the plate to move.

- `target_carrier_name` - String identifier of the target plate carrier to move the plate to.

- `target_plate_slot` - The plate slot on the carrier to move the tip box to.

- Include a **0.5s delay**

**Hint:** You may find assigning/unassigning resources from the STAR carriers a bit different. You should check out their documentation for [assigning](https://github.com/PyLabRobot/pylabrobot/blob/7234de0ae521c8be9cd7b52cdc881eb3bdb82254/pylabrobot/resources/carrier.py#L78) and [unassigning](https://github.com/PyLabRobot/pylabrobot/blob/7234de0ae521c8be9cd7b52cdc881eb3bdb82254/pylabrobot/resources/carrier.py#L83) resources

Comment your code

In [ ]:
class Exercise_2B_Gripper(Exercise_2A_Gripper):
    def move_plate(self,
                    input_carrier_name: str,
                    plate_name: str,
                    target_carrier_name: str,
                    target_plate_slot: int):
        ... # YOUR CODE HERE

Great, now use the gripper you just implemented to move every plate from the first carrier to the second in **three lines of code or less (excluding comments and blank lines)**

Comment your code

In [ ]:
# define gripper
gripper = Exercise_2B_Gripper(lh)

... # YOUR CODE HERE

**2.C.** Rail Gripper

Great. Now we need a functionality to **add or move** plate carriers to different rails on the deck. Implement two functions:

1. `move_carrier_to_rails()`

    - `carrier_name` - String name of the carrier to move.

    - `target_rail` - Integer of the target rails to move the carrier to.

2. `add_carrier()`

    - `type` - Either `tip_carrier`, `plate_carrier`, or `trough_carrier` to add to the deck.
    
    - `carrier_name` - Name of the carrier being added.

    - `target_rail` - Integer of the target rails to add the carrier to.

Comment your code

In [ ]:
# imports
from pylabrobot.resources import ... # YOUR CODE HEREA

class Exercise_2C_Gripper(Exercise_2B_Gripper):
    def move_carrier_to_rails(self,
                              carrier_name: str,
                              target_rails: int
    ):
        ... # YOUR CODE HERE
    
    def add_carrier(self,
                    carrier_type: str,
                    carrier_name: str,
                    target_rails: int):
        ... # YOUR CODE HERE

Now use your new gripper to move the tip rack on the left to the **farthest possible rails on the right it will fit without running into the trash**.

Also, add a new tip carrier to rails 5. Do this in **three lines or less (excluding comments and empty lines)**

Comment your code

In [ ]:
# define gripper
gripper = Exercise_2C_Gripper(lh)

... # YOUR CODE HERE

**2.D.** Add Lids

Finally, let's write a function that will add lids **from the bottom plate upwards** given an input **carrier** and **number of plates to add lids to**. Specifically it should input:

- `carrier_name` - The carrier name to add lids to plates on.

- `num_lids` - The number of plates to add lids to. Must be less than or equal to the number of plates on the carrier.

Comment your code

In [ ]:
class Exercise_2D_Gripper(Exercise_2C_Gripper):
    def add_lids(self,
                 carrier_name: str,
                 num_lids: int):

        ... # YOUR CODE HERE

Using this code, go ahead and add lids to the first 4 plates on the carrier currently with the plates. Then move only the plates with lids over to the other carrier.

Comment your code.

In [ ]:
# define a gripper class
gripper = Exercise_2D_Gripper(lh)
... # YOUR CODE HERE

**2.E.** Plate Staircase 2.0

Remember the plate staircase exercise from workshop 1? We are going to try to do it again, this time with our custom gripper and some unique constraints:

- You are **not allowed** to add plates or tip boxes to the deck via any method. You will need to add extra carriers.

- You must make all operations on the deck via **gripper class methods** (i.e. only method calls should be of the form `gripper.method_name()`). You are still allowed to use external lists for names and moves.

- You are **not allowed** to remove **ANY** labware once added to the deck.

Given the following incomplete deck:

In [ ]:
# DO NOT MODIFY
from pylabrobot.resources import PLT_CAR_L5AC_A00, TIP_CAR_480_A00, HTF

def make_incomplete_staircase():
    deck = STARDeck()
    plate_rails = [0, 7]
    plate_plates = [4, 5]
    tip_rails = [39, 48]
    for i, (plate_rails, plate_plates) in enumerate(zip(plate_rails, plate_plates)):
        plate_carrier = PLT_CAR_L5AC_A00(f"plate_carrier_{i}")
        for j in range(plate_plates):
            plate_carrier[j] = corning_96_wellplate_360ul_flat(name = f"plate_{i}_{j}")
        deck.assign_child_resource(plate_carrier, rails = plate_rails)
    for i, tip_rails in enumerate(tip_rails):
        tip_carrier = TIP_CAR_480_A00(f"tip_carrier_{i}")
        for j in range(3):
            tip_carrier[j] = HTF(f"tip_rack_{i}_{j}")
        deck.assign_child_resource(tip_carrier, rails = tip_rails)
    return deck
    
deck = make_incomplete_staircase()
lh = await visualize_deck(deck, LiquidHandlerChatterboxBackend())

You should create a protocol in `exercise_2e_protocol()` that takes in your `gripper` alone and is able to create a plate staircase such that:

- Plate and tip carriers should alternate, starting with a tip carrier at rail 7
    
    - To put it explicitly, carriers should be in order of T-P-T-P-T-P where T is a tip carrier and P is a plate carrier.

- There should be a gap of at least 1 rail between each carrier.

- The leftmost plate staircase should be a tip carrier with 0 tip racks.

- The rightmorst plate staircase should be a plate carrier with 5 plates.

    - To put it explictly, you should have 0-1-2-3-4-5 items on the 6 carriers.

- Progressively more plates should be added as you move left to right, creating a "staircase" shape.

- Every plate carrier should have lids on **all plates but the topmost plate**. (i.e. a carrier with 4 plates should have the bottom 3 covered with lids)

- You should be able to achieve this in **16 lines of code or less (excluding empty lines and comments)**

- Comment your code.

In [ ]:
# define gripper
gripper = Exercise_2D_Gripper(lh)

def exercise_2e_protocol(gripper):
    ... # YOUR CODE HERE

# run protocol
exercise_2e_protocol(gripper)

---

**TO-DO:** Submission Instructions. In the cell below, we provide a template for your .txt file submission. Please copy-paste the relevant parts of your code into the designated sections below and submit as a .txt file named `exercise_2.txt`

**TO-DO:** Please run your code, starting from the **blank deck** all the way through the addition of lids to the plates and final deck layout up to, but not including part E, record the GIF, and submit as a .gif file named `exercise_2.gif`

**TO-DO:** Please run your code, starting from the **mis-configured staircase deck** all the way through the solution of the staircase with lids, record the GIF, and submiit as a .gif file named `exercise_2_part_e.gif`

---

In [ ]:
# Exercise 2. All Imports
# --- YOUR CODE HERE ---

# CLASS METHODS SECTION
# Exercise 2.A. Initial Gripper
class Exercise_2A_Gripper:
    def __init__(self, lh: LiquidHandler):
        ... # YOUR CODE HERE

    def get_labware_by_name(self, name: str):
        ... # YOUR CODE HERE

# Exercise 2.B. Move Labware
class Exercise_2B_Gripper(Exercise_2A_Gripper):
    def move_labware(self,
                    input_carrier_name: str,
                    labware_name: str,
                    target_carrier_name: str,
                    target_labware_slot: int):
        ... # YOUR CODE HERE

# Exercise 2.C. Carrier Movement
class Exercise_2C_Gripper(Exercise_2B_Gripper):
    def move_carrier_to_rails(self,
                              carrier_name: str,
                              target_rails: int
    ):
        ... # YOUR CODE HERE

    def add_carrier(self,
                    carrier_type: str,
                    carrier_name: str,
                    target_rails: int):
        ... # YOUR CODE HERE

# Exercise 2.D. Add Lids
class Exercise_2D_Gripper(Exercise_2C_Gripper):
    def add_lids(self,
                 carrier_name: str,
                 num_lids: int):
        ... # YOUR CODE HERE

# CODE BLOCKS SECTION
# Exercise 2.B. Move Plates
gripper = Exercise_2B_Gripper(lh = lh)
... # YOUR CODE HERE

# Exercise 2.C. Move/Add Carriers
gripper = Exercise_2C_Gripper(lh = lh)
... # YOUR CODE HERE

# Exercise 2.D. Add Lids
gripper = Exercise_2D_Gripper(lh = lh)
... # YOUR CODE HERE

# Exercise 2.E. Staircase Fixing
gripper = Exercise_2D_Gripper(lh)
time.sleep(1)

def exercise_2e_protocol(gripper):

    ... # YOUR CODE HERE

exercise_2e_protocol(gripper)

# Save as exercise_2.txt and submit. Don't forget to submit exercise_2.gif AND exercise_2_part_2.gif as well.

---

#### Conclusion

That's all for workshop 3! Double check that you have submitted a `.gif` file and `.txt` file for each exercise. You should have submitted:

- `exercise_1.txt` following the schema shown above

- `exercise_2.txt` following the schema shown above

- `exercise_1.gif` showing the full protocol in exercise 1, starting with the blank OT-2 deck.

- `exercise_2.gif` showing the full protocol in exercise 2, except part E, starting with the blank STAR deck.

- `exercise_2_part_e.gif` showing the Protocol in exercise 2, PART E ONLY, starting with the misconfigured staircase deck.

**EVERY CODE BLOCK SHOULD HAVE WELL-WRITTEN COMMENTS**

If you are still feeling unsure on deck setup, please **reach out to the teaching team**, contact info for whom can be found in the .`README.md` file on the [class GitHub](https://github.com/chory-lab/bme590-fall-2025)

---